In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score

## create binary target

In [2]:
k = 20
score_levels = [0.1,0.2,0.3,0.5,0.7,0.8,0.9]

# start, end, step = 0, 0.5, 0.01
# score_levels = [start + i * step for i in range(int((end - start) / step) + 1)]

In [3]:
dff = pd.read_csv(f'df_test_results_k{k}.csv', encoding='utf-8', index_col=0)
test_df = pd.read_csv(f'test_df.csv', encoding='utf-8', index_col=0)

In [4]:
def new_score_column(df, score_level):
    new_col = f"score_{score_level}"
    df[new_col] = df['target'].apply(lambda x: 1 if x > score_level else 0)
    return df

def create_table_like_submission(df,score):
    df['id'] = df.apply(lambda row: f"{int(row['user_id'])};{int(row['cart_id'])}", axis=1)
    df_new = df[['id',f'score_{score}']]
    df_new = df_new.rename(columns={f'score_{score}': 'target'})
    return df_new

def create_table_like_submission_from_test(test_df):
    test_df['id'] = test_df.apply(lambda row: f"{int(row['user_id'])};{int(row['cart'])}", axis=1)
    test_df['target'] = 1
    submit_df=test_df[['id','target']]
    submit_df
    return submit_df

In [5]:
for score_level in score_levels:
    new_score_column(dff, score_level)

submit_df = create_table_like_submission_from_test(test_df)

dfs = {} 
for score in score_levels: 
    dfs[score] = create_table_like_submission(dff, score=score)

In [6]:
dfs[0.5]

,id,target
0,19999;381,0
1,19999;342,0
2,19999;350,0
3,19999;352,0
4,19999;58,0
...,...,...
399995,0;50,1
399996,0;74,1
399997,0;48,1
399998,0;357,1


# f1 score

In [7]:
import numpy as np

score = 0.2
# ничего правильно не предсказывает
df_merged = pd. merge(submit_df, dfs[score], on='id', how='outer' )
df_merged.sort_values(by='id').head(30)

,id,target_x,target_y
657895,0;146,NaN,0.0
657891,0;154,NaN,0.0
657904,0;158,NaN,1.0
657902,0;159,NaN,1.0
657897,0;160,NaN,0.0
657898,0;167,NaN,0.0
10,0;169,1.0,NaN
1,0;170,1.0,NaN
657896,0;179,NaN,0.0
657908,0;357,NaN,1.0


In [8]:
# заменяем наны нулями - так как не предсказано или не было
df_merged.fillna(0, inplace=True)

In [9]:
true_labels = df_merged['target_x']
predicted_labels = df_merged['target_y']

f1 = f1_score(true_labels, predicted_labels)
f1

0.062106919717142016

In [10]:
# в цикле проделываем для разных скоров
f1_scores = {}

for score in score_levels: 
    df_merged = pd.DataFrame()
    # ничего правильно не предсказывает
    df_merged = pd.merge(submit_df, dfs[score], on='id', how='outer' )
    df_merged.sort_values(by='id').head(30)
    # заменяем наны нулями - так как не предсказано или не было
    df_merged.fillna(0, inplace=True)
    f1_scores[score] = f1_score(df_merged['target_x'], df_merged['target_y'])
   
print(f1_scores)

{0.0: 0.06803048700807691, 0.01: 0.06838830834826926, 0.02: 0.06879089607472022, 0.03: 0.06882214251561448, 0.04: 0.06927120260866805, 0.05: 0.06959429491996093, 0.06: 0.06981697478216772, 0.07: 0.07004131301864283, 0.08: 0.07017774831240195, 0.09: 0.07017137710574387, 0.1: 0.07008452332217593, 0.11: 0.06991017501143361, 0.12: 0.06968220003170074, 0.13: 0.06929134386311397, 0.14: 0.06873998272898992, 0.15: 0.06793589776888911, 0.16: 0.066936355538656, 0.17: 0.06584231001276458, 0.18: 0.06466094783140602, 0.19: 0.06334486650507891, 0.2: 0.062106919717142016, 0.21: 0.06057467398449802, 0.22: 0.05919856224854391, 0.23: 0.057806109986052776, 0.24: 0.05653035078637628, 0.25: 0.0553535599197422, 0.26: 0.05399710421736854, 0.27: 0.05291192442937101, 0.28: 0.051668569550764876, 0.29: 0.05062136489283028, 0.3: 0.049562914649308225, 0.31: 0.048627864240861865, 0.32: 0.04782790400985417, 0.33: 0.047208499394962085, 0.34: 0.04647254557065552, 0.35000000000000003: 0.04592223154191715, 0.36: 0.04538